In [0]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

In [0]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    #print(image)
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

  

def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap

#changed mapping such that cars and buses are mapped to a pixel with color
#values [0,0,0]. The rest of the colormap is filled with 1s.
def create_pascal_label_colormap_1():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.ones((256, 3), dtype=int)
  colormap[7,:] = 0 
  colormap[6,:] = 0
  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap_1()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [0]:
#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
#!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.6 GB  | Proc size: 303.6 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
tarball_path = "/content/drive/My Drive/comma_challenge/models/deeplabv3_pascal_trainval_2018_01_04.tar.gz"
dlmodel = DeepLabModel(tarball_path)
os.chdir("/content/drive/My Drive/comma_challenge/")

In [0]:
def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

In [0]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
newWidth, newHeight = 220, 74
top = 0
bottom = 370
left = 0
right = 640

#paths to video to be used and the directory to store the optical flow images
#currently set to test video
path_to_vid = "/content/drive/My Drive/comma_challenge/data/test.mp4"
path_to_flow = "/content/drive/My Drive/comma_challenge/data/flow_imgs_test_cars"

cap = cv2.VideoCapture(path_to_vid)
ret, frame1 = cap.read()
ret, frame2 = cap.read()


prvs = cv2.cvtColor(frame1[top:bottom, left:right], cv2.COLOR_BGR2RGB)
prvs, prvs_seg = dlmodel.run(Image.fromarray(prvs.astype('uint8'), 'RGB'))
prvs_seg_img = label_to_color_image(prvs_seg)
prvs = np.array(prvs)
prvs = cv2.cvtColor(prvs,cv2.COLOR_RGB2GRAY)


curr = cv2.cvtColor(frame2[top:bottom, left:right], cv2.COLOR_BGR2RGB)
curr, curr_seg = dlmodel.run(Image.fromarray(curr.astype('uint8'), 'RGB'))
curr_seg_img = label_to_color_image(curr_seg)
curr = np.array(curr)
curr = cv2.cvtColor(curr,cv2.COLOR_RGB2GRAY)

i = 2

In [0]:
while(1):
    ret, frame3 = cap.read()
    if(not ret):
        break
    
    next_ = cv2.cvtColor(frame3[top:bottom, left:right], cv2.COLOR_BGR2RGB)
    next_, next_seg = dlmodel.run(Image.fromarray(next_.astype('uint8'), 'RGB'))
    next_seg_img = label_to_color_image(next_seg)
    next_ = np.array(next_)
    next_ = cv2.cvtColor(next_,cv2.COLOR_RGB2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvs,next_, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    hsv2 = draw_hsv(flow)
    hsv2 = hsv2 * curr_seg_img
    hsv2 = cv2.resize(hsv2.astype('float32'), (newWidth, newHeight), interpolation=cv2.INTER_AREA)
    
    name = ""
    if i < 10:
        name = "0000"
    elif i < 100:
        name = "000"
    elif i < 1000:
        name = "00"
    elif i < 10000:
        name = "0"
    else:
        name = ""
    
    name += str(i)
    cv2.imwrite(os.path.join(path_to_flow, "hsv_" + name + ".png"),hsv2)
    
    prvs = curr
    prvs_seg_img = curr_seg_img
    curr = next_
    curr_seg_img = next_seg_img
    if(i % 100 == 0):
      print(i)
    i += 1
cap.release()
cv2.destroyAllWindows()

In [0]:
print(i)

10798


In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/")
os.chdir("./data")

In [0]:
!mkdir flow_imgs_test_cars

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
checkpoints_withtest	      model_with_dropout_withtest.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test_bothflow_best.txt
flow_imgs_train_cars	      test.mp4
flow_test_imgs		      test.txt
flow_test_imgs.tar	      test_vert_best.txt
horz_flow_imgs		      test_vert.txt
horz_flow_imgs.tar	      test_withtest_best.txt
horz_flow_test_imgs	      train.mp4
horz_flow_test_imgs.tar       train.txt
logs			      vert_flow_imgs
model_with_dropout_both.h5    vert_flow_imgs.tar
model_with_dropout_both.json  vert_flow_test_imgs
model_with_dropout.h5	      vert_flow_test_imgs.tar
